In [ ]:
import numpy as np
import pandas as pd
from gnews import GNews
import yfinance as yf
from datetime import datetime, timedelta

In [48]:
def extract_economic_news(language='fr', max_results=50, period='7d', topic='ECONOMY'):
    google_news = GNews(language=language, max_results=max_results, period=period)
    news = google_news.get_news_by_topic(topic)
    title_news = [article['title'] for article in news]
    return news,title_news

In [ ]:
french_economy, french_title = extract_economic_news()
german_economy, german_title = extract_economic_news(language='de')

Prix des carburants : six questions sur une hausse qui fait polémique - Les Echos


In [41]:
df_oat = pd.read_csv("Rendement de l'Obligation France 10 ans - Données Historiques (1).csv")
df_bund = pd.read_csv("Rendement de l'Obligation Allemagne 10 ans - Données Historiques.csv")